Dataset:
https://www.kaggle.com/valentynsichkar/traffic-signs-preprocessed



In [ ]:
from PIL import Image
import tensorflow as tf

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
from math import sqrt, ceil
from timeit import default_timer as timer

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#link = 'https://drive.google.com/file/d/1GLzsLInYuWebxs2GwIaBVrXJS9VKEhYR/view?usp=sharing' # The shareable link

In [ ]:
id2 = "13Wrx_D9MsD2beWX6VZ197JIeX8hU9Okn"

In [ ]:
#id = "1GLzsLInYuWebxs2GwIaBVrXJS9VKEhYR"

In [ ]:
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('data0.pickle') 
#data = pickle.load(open("data0.pickle","rb")) 

In [ ]:
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('data1.pickle') 
data = pickle.load(open("data1.pickle","rb")) 

#### Pre procesamiento

In [ ]:
# Preparing y_train and y_validation for using in Keras
data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

In [ ]:
def one_hottie(labels,C):
    One_hot_matrix = tf.one_hot(labels,C)
    return tf.keras.backend.eval(One_hot_matrix)

In [ ]:
Y_test = data['y_test']
Y_test = one_hottie(Y_test, 43)

#### Modelo de referencia

In [ ]:
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=80000,
    decay_rate=1,
    staircase=True)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 12, 12, 64)        0

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 14s 17ms/step - loss: 0.8502 - accuracy: 0.7620 - val_loss: 0.3241 - val_accuracy: 0.9088
Epoch 2/10
870/870 [==============================] - 14s 16ms/step - loss: 0.0957 - accuracy: 0.9727 - val_loss: 0.1923 - val_accuracy: 0.9587
Epoch 3/10
870/870 [==============================] - 14s 16ms/step - loss: 0.0496 - accuracy: 0.9858 - val_loss: 0.1697 - val_accuracy: 0.9630
Epoch 4/10
870/870 [==============================] - 14s 16ms/step - loss: 0.0375 - accuracy: 0.9886 - val_loss: 0.1829 - val_accuracy: 0.9594
Epoch 5/10
870/870 [==============================] - 14s 16ms/step - loss: 0.0301 - accuracy: 0.9910 - val_loss: 0.1914 - val_accuracy: 0.9621
Epoch 6/10
870/870 [==============================] - 14s 16ms/step - loss: 0.0228 - accuracy: 0.9926 - val_loss: 0.1493 - val_accuracy: 0.9683
Epoch 7/10
870/870 [==============================] - 14s 16ms/step - loss: 0.0214 - accuracy: 0.9936 - val_loss: 0.1231 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.3491 - accuracy: 0.9511


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.9513916, shape=(), dtype=float32)


### Modelo con Dropout

##### Dropout antes de pooling

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 16s 18ms/step - loss: 0.9797 - accuracy: 0.7180 - val_loss: 0.2045 - val_accuracy: 0.9397
Epoch 2/10
870/870 [==============================] - 15s 18ms/step - loss: 0.1811 - accuracy: 0.9451 - val_loss: 0.1501 - val_accuracy: 0.9551
Epoch 3/10
870/870 [==============================] - 15s 18ms/step - loss: 0.1050 - accuracy: 0.9685 - val_loss: 0.0944 - val_accuracy: 0.9730
Epoch 4/10
870/870 [==============================] - 16s 18ms/step - loss: 0.0786 - accuracy: 0.9763 - val_loss: 0.1246 - val_accuracy: 0.9658
Epoch 5/10
870/870 [==============================] - 15s 18ms/step - loss: 0.0636 - accuracy: 0.9809 - val_loss: 0.0850 - val_accuracy: 0.9739
Epoch 6/10
870/870 [==============================] - 15s 18ms/step - loss: 0.0525 - accuracy: 0.9844 - val_loss: 0.0696 - val_accuracy: 0.9810
Epoch 7/10
870/870 [==============================] - 15s 18ms/step - loss: 0.0491 - accuracy: 0.9853 - val_loss: 0.0593 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1670 - accuracy: 0.9682


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.96938115, shape=(), dtype=float32)


##### Dropout entre no linealidad y convolucion

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 17s 20ms/step - loss: 1.2129 - accuracy: 0.6371 - val_loss: 0.2846 - val_accuracy: 0.9279
Epoch 2/10
870/870 [==============================] - 17s 20ms/step - loss: 0.2508 - accuracy: 0.9204 - val_loss: 0.1212 - val_accuracy: 0.9624
Epoch 3/10
870/870 [==============================] - 17s 20ms/step - loss: 0.1530 - accuracy: 0.9529 - val_loss: 0.0962 - val_accuracy: 0.9705
Epoch 4/10
870/870 [==============================] - 17s 20ms/step - loss: 0.1125 - accuracy: 0.9649 - val_loss: 0.0676 - val_accuracy: 0.9787
Epoch 5/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0908 - accuracy: 0.9727 - val_loss: 0.0578 - val_accuracy: 0.9830
Epoch 6/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0811 - accuracy: 0.9750 - val_loss: 0.0611 - val_accuracy: 0.9803
Epoch 7/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0664 - accuracy: 0.9793 - val_loss: 0.0576 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1354 - accuracy: 0.9695


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.97193426, shape=(), dtype=float32)


##### Dropout antes de la convolucion

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 16s 19ms/step - loss: 1.4363 - accuracy: 0.5581 - val_loss: 0.4526 - val_accuracy: 0.8594
Epoch 2/10
870/870 [==============================] - 16s 19ms/step - loss: 0.2837 - accuracy: 0.9059 - val_loss: 0.2170 - val_accuracy: 0.9497
Epoch 3/10
870/870 [==============================] - 16s 19ms/step - loss: 0.1529 - accuracy: 0.9505 - val_loss: 0.1293 - val_accuracy: 0.9612
Epoch 4/10
870/870 [==============================] - 16s 19ms/step - loss: 0.1091 - accuracy: 0.9648 - val_loss: 0.1437 - val_accuracy: 0.9649
Epoch 5/10
870/870 [==============================] - 16s 19ms/step - loss: 0.0878 - accuracy: 0.9714 - val_loss: 0.0861 - val_accuracy: 0.9739
Epoch 6/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0760 - accuracy: 0.9754 - val_loss: 0.1153 - val_accuracy: 0.9723
Epoch 7/10
870/870 [==============================] - 16s 19ms/step - loss: 0.0630 - accuracy: 0.9799 - val_loss: 0.0610 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1874 - accuracy: 0.9673


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.9682754, shape=(), dtype=float32)


### Modelo con Batchnormalization

##### Batchnormalization antes de la convolucion

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 17s 20ms/step - loss: 0.2445 - accuracy: 0.9353 - val_loss: 0.1268 - val_accuracy: 0.9651
Epoch 2/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0258 - accuracy: 0.9921 - val_loss: 0.0946 - val_accuracy: 0.9771
Epoch 3/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0190 - accuracy: 0.9942 - val_loss: 0.0900 - val_accuracy: 0.9735
Epoch 4/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0164 - accuracy: 0.9947 - val_loss: 0.1234 - val_accuracy: 0.9753
Epoch 5/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0131 - accuracy: 0.9960 - val_loss: 0.0644 - val_accuracy: 0.9853
Epoch 6/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.0941 - val_accuracy: 0.9773
Epoch 7/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0110 - accuracy: 0.9966 - val_loss: 0.0937 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.2012 - accuracy: 0.9702


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.97178674, shape=(), dtype=float32)


##### Batchnormalization entre no linealidad y convolucion

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 17s 19ms/step - loss: 0.3056 - accuracy: 0.9262 - val_loss: 0.1250 - val_accuracy: 0.9658
Epoch 2/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0228 - accuracy: 0.9935 - val_loss: 0.1551 - val_accuracy: 0.9533
Epoch 3/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0153 - accuracy: 0.9954 - val_loss: 0.0870 - val_accuracy: 0.9805
Epoch 4/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0118 - accuracy: 0.9962 - val_loss: 0.1374 - val_accuracy: 0.9701
Epoch 5/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0123 - accuracy: 0.9962 - val_loss: 0.0828 - val_accuracy: 0.9825
Epoch 6/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0077 - accuracy: 0.9974 - val_loss: 0.0882 - val_accuracy: 0.9771
Epoch 7/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0087 - accuracy: 0.9974 - val_loss: 0.0580 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.2063 - accuracy: 0.9614


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.96255404, shape=(), dtype=float32)


### Modelo con ambos

##### D/BNx6

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 20s 24ms/step - loss: 0.6967 - accuracy: 0.7923 - val_loss: 0.1144 - val_accuracy: 0.9658
Epoch 2/10
870/870 [==============================] - 20s 23ms/step - loss: 0.1355 - accuracy: 0.9578 - val_loss: 0.0649 - val_accuracy: 0.9796
Epoch 3/10
870/870 [==============================] - 20s 23ms/step - loss: 0.0877 - accuracy: 0.9724 - val_loss: 0.0435 - val_accuracy: 0.9880
Epoch 4/10
870/870 [==============================] - 20s 23ms/step - loss: 0.0706 - accuracy: 0.9780 - val_loss: 0.0581 - val_accuracy: 0.9830
Epoch 5/10
870/870 [==============================] - 20s 23ms/step - loss: 0.0586 - accuracy: 0.9817 - val_loss: 0.0473 - val_accuracy: 0.9862
Epoch 6/10
870/870 [==============================] - 20s 23ms/step - loss: 0.0501 - accuracy: 0.9845 - val_loss: 0.0419 - val_accuracy: 0.9909
Epoch 7/10
870/870 [==============================] - 20s 23ms/step - loss: 0.0469 - accuracy: 0.9857 - val_loss: 0.0426 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1046 - accuracy: 0.9752


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.9766609, shape=(), dtype=float32)


##### Dropout fijo/ 2D/1BN/2D/1BN

In [ ]:
model = tf.keras.Sequential()
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 18s 20ms/step - loss: 0.5178 - accuracy: 0.8498 - val_loss: 0.0966 - val_accuracy: 0.9719
Epoch 2/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0679 - accuracy: 0.9786 - val_loss: 0.0950 - val_accuracy: 0.9726
Epoch 3/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0427 - accuracy: 0.9868 - val_loss: 0.0774 - val_accuracy: 0.9751
Epoch 4/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0361 - accuracy: 0.9884 - val_loss: 0.1157 - val_accuracy: 0.9760
Epoch 5/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0298 - accuracy: 0.9907 - val_loss: 0.0706 - val_accuracy: 0.9807
Epoch 6/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0259 - accuracy: 0.9915 - val_loss: 0.0666 - val_accuracy: 0.9834
Epoch 7/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0226 - accuracy: 0.9929 - val_loss: 0.0600 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1720 - accuracy: 0.9641


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.9656271, shape=(), dtype=float32)


##### Dropout fijo/ 2BN/1D/2BN/1D

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 17s 20ms/step - loss: 0.4713 - accuracy: 0.8620 - val_loss: 0.1263 - val_accuracy: 0.9642
Epoch 2/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0875 - accuracy: 0.9732 - val_loss: 0.1003 - val_accuracy: 0.9735
Epoch 3/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0539 - accuracy: 0.9835 - val_loss: 0.0675 - val_accuracy: 0.9823
Epoch 4/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0485 - accuracy: 0.9855 - val_loss: 0.0756 - val_accuracy: 0.9800
Epoch 5/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0363 - accuracy: 0.9890 - val_loss: 0.0997 - val_accuracy: 0.9814
Epoch 6/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0318 - accuracy: 0.9902 - val_loss: 0.1379 - val_accuracy: 0.9721
Epoch 7/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0292 - accuracy: 0.9912 - val_loss: 0.0575 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1141 - accuracy: 0.9808


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.98214424, shape=(), dtype=float32)


##### Dropout fijo/ 3BN/3D

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 17s 19ms/step - loss: 0.6600 - accuracy: 0.8034 - val_loss: 0.1082 - val_accuracy: 0.9653
Epoch 2/10
870/870 [==============================] - 16s 19ms/step - loss: 0.1202 - accuracy: 0.9638 - val_loss: 0.0837 - val_accuracy: 0.9760
Epoch 3/10
870/870 [==============================] - 16s 19ms/step - loss: 0.0788 - accuracy: 0.9771 - val_loss: 0.0534 - val_accuracy: 0.9855
Epoch 4/10
870/870 [==============================] - 16s 19ms/step - loss: 0.0620 - accuracy: 0.9816 - val_loss: 0.0803 - val_accuracy: 0.9785
Epoch 5/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0516 - accuracy: 0.9851 - val_loss: 0.0414 - val_accuracy: 0.9882
Epoch 6/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0482 - accuracy: 0.9862 - val_loss: 0.0448 - val_accuracy: 0.9891
Epoch 7/10
870/870 [==============================] - 17s 19ms/step - loss: 0.0443 - accuracy: 0.9875 - val_loss: 0.0663 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.1431 - accuracy: 0.9756


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.9773766, shape=(), dtype=float32)


##### Dropout fijo/ 3D/3BN

In [ ]:
model = tf.keras.Sequential()
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(32,32,3)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3,strides=2))
#model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(padding = 'same',strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
result = model.fit(x=data['x_train'],y=data['y_train'],batch_size=100,epochs=10,validation_data = (data['x_validation'], data['y_validation']),verbose=1,shuffle=False,initial_epoch=0)

Epoch 1/10
870/870 [==============================] - 18s 21ms/step - loss: 0.4394 - accuracy: 0.8758 - val_loss: 0.1702 - val_accuracy: 0.9483
Epoch 2/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0511 - accuracy: 0.9840 - val_loss: 0.1189 - val_accuracy: 0.9726
Epoch 3/10
870/870 [==============================] - 18s 20ms/step - loss: 0.0331 - accuracy: 0.9895 - val_loss: 0.1185 - val_accuracy: 0.9712
Epoch 4/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0282 - accuracy: 0.9912 - val_loss: 0.1054 - val_accuracy: 0.9748
Epoch 5/10
870/870 [==============================] - 18s 20ms/step - loss: 0.0236 - accuracy: 0.9925 - val_loss: 0.1070 - val_accuracy: 0.9769
Epoch 6/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0200 - accuracy: 0.9932 - val_loss: 0.1471 - val_accuracy: 0.9732
Epoch 7/10
870/870 [==============================] - 17s 20ms/step - loss: 0.0153 - accuracy: 0.9952 - val_loss: 0.1162 - val_accuracy:

In [ ]:
valid = model.evaluate(data['x_test'],Y_test,verbose=2)

395/395 - 2s - loss: 0.2094 - accuracy: 0.9583


In [ ]:
y_new = model.predict(data['x_test'])
print(f1_m(Y_test, y_new))

tf.Tensor(0.95977753, shape=(), dtype=float32)


### <a id="segundo"></a>
## 2.Challenge Kaggle
